In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [108]:
reward = [env.P[i][0][0][2] for i in range(len(env.P))]


[0.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.0]

In [109]:
transition_matrix = np.zeros((16,16))
for i in range(len(env.P)):
    for j in range(len(env.P[i])):
        transition_matrix[i][env.P[i][j][0][1]] = 0.25

In [110]:
transition_matrix[0][0] = 1.0
transition_matrix[15][15] = 1.0

In [111]:
np.dot(transition_matrix, reward)

array([ 0.  , -0.75, -1.  , -0.75, -0.75, -1.  , -1.  , -1.  , -1.  ,
       -1.  , -1.  , -0.75, -0.75, -1.  , -0.75,  0.  ])

In [107]:
np.ones([env.nS, env.nA]) / env.nA

array([[ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25]])

In [112]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    updated_V = np.zeros(env.nS)
    
    # Create Transition Matrix - probably needs refactoring
    transition_matrix = np.zeros((env.nS,env.nS))
    for i in range(len(env.P)):
        for j in range(len(env.P[i])):
            transition_matrix[i][env.P[i][j][0][1]] = 0.25 # uniform random policy
    
    transition_matrix[0][0] = 1.0
    transition_matrix[15][15] = 1.0
    
    while True:
        # TODO: Implement!
        rewards = [env.P[i][0][0][2] for i in range(len(env.P))]
        updated_V = rewards + discount_factor*(np.dot(transition_matrix, V))
        delta = abs(updated_V - V)
        
        print (V)
        
        V = updated_V
        
        if all(i < theta for i in delta):
            break
    return np.array(V)

In [113]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.   -0.75 -1.   -0.75 -0.75 -1.   -1.   -1.   -1.   -1.   -1.   -0.75
 -0.75 -1.   -0.75  0.  ]
[ 0.     -1.4375 -1.875  -1.4375 -1.4375 -1.875  -2.     -1.875  -1.875  -2.
 -1.875  -1.4375 -1.4375 -1.875  -1.4375  0.    ]
[ 0.       -2.046875 -2.6875   -2.046875 -2.046875 -2.71875  -2.875    -2.6875
 -2.6875   -2.875    -2.71875  -2.046875 -2.046875 -2.6875   -2.046875  0.      ]
[ 0.         -2.61328125 -3.4140625  -2.60546875 -2.61328125 -3.4609375
 -3.703125   -3.4140625  -3.4140625  -3.703125   -3.4609375  -2.61328125
 -2.60546875 -3.4140625  -2.61328125  0.        ]
[ 0.         -3.12207031 -4.08398438 -3.10839844 -3.12207031 -4.15820312
 -4.4375     -4.08398438 -4.08398438 -4.4375     -4.15820312 -3.12207031
 -3.10839844 -4.08398438 -3.12207031  0.        ]
[ 0.         -3.59106445 -4.68798828 -3.5690918  -3.59106445 -4.77978516
 -5.12109375 -4.68798828 -4.68798828 -5.12109375 -4.77978516 -3.59106445
 -3.569091

In [114]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

AssertionError: 
Arrays are not almost equal to 2 decimals

(mismatch 87.5%)
 x: array([  0.  ,  -8.37, -10.89,  -8.26,  -8.37, -11.21, -12.05, -10.89,
       -10.89, -12.05, -11.21,  -8.37,  -8.26, -10.89,  -8.37,   0.  ])
 y: array([  0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22,
       -20, -14,   0])